## Preprocess GSL isolated, select words with freq > 20

In [25]:
from collections import Counter
import pandas as pd
df = pd.read_csv("/Users/liweijie/Downloads/Greek_isolated/GSL_isol/iso_annotations.csv",sep='|')
vocab = []
for i in range(len(df)):
    word = df.iloc[i,1].strip()
    vocab.append(word)
vocab = Counter(vocab)
vocab = sorted(vocab.items(),key = lambda item:item[1],reverse=True)
wordToId = {}
select_vocab = []
count = 0
for word in vocab:
    if word[1] >= 20:
        try:
            int(word[0])
        except:
            if word[0] == '60+3':
                continue
            select_vocab.append(word)
            wordToId[word[0]] = count
            count += 1
select_vocab,wordToId
select_vocab[210]

('ΔΕΗ', 35)

## Generate trainval, test & trainvaltest list, Account health, kep & police vocab

In [23]:
import os
import os.path as osp
import numpy
import json
n_class = len(select_vocab) #315
n_base = 210
n_novel = 105
C_base = range(n_base)
C_novel = range(n_base,n_class)
health_vocab = []
kep_vocab = []
police_vocab = []
# open list files
trainvaltest_csv = open('../csv/gsl/trainvaltest.csv','w')
trainvaltest_csv.write('foldername,label\n')
trainval_csv = open('../csv/gsl/trainval.csv','w')
trainval_csv.write('foldername,label\n')
test_csv = open('../csv/gsl/test.csv','w')
test_csv.write('foldername,label\n')
container = []
for i in range(len(df)):
    path = df.iloc[i,0]
    word = df.iloc[i,1].strip()
    if word in wordToId.keys():
        id = wordToId[word]
        container.append([path,id])
        if ('health' in path) and (not id in health_vocab):
            health_vocab.append(id)
        if ('kep' in path) and (not id in kep_vocab):
            kep_vocab.append(id)
        if ('police' in path) and (not id in police_vocab):
            police_vocab.append(id)
container = sorted(container,key = lambda item:item[1])
for item in container:
    path, id = item
    record = path.strip()+'/keypoints.npy' + ',' + str(id) + '\n'
    if id in C_base:
        trainval_csv.write(record)
    else:
        test_csv.write(record)
    trainvaltest_csv.write(record)
health_vocab.sort()
kep_vocab.sort()
police_vocab.sort()
my_dict = {'health':health_vocab,
           'kep':kep_vocab,
           'police':police_vocab}
json_fname = '../csv/gsl/vocab_category.json'
json_str = json.dumps(my_dict, indent=4)
with open(json_fname, 'w') as json_file:
    json_file.write(json_str)